In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 101.0 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python3
# ----------------------------
# Main CNN model for binarized MNIST using 2D convolutions on 28x28x1 binary images
# ----------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from pennylane.data import load as qml_load

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


In [ ]:
# ----------------------------
# Load binarized MNIST
# ----------------------------
def load_binarized_mnist():
    [ds] = qml_load("other", name="binarized-mnist")

    x_train = ds.train["inputs"].astype("float32")
    y_train = ds.train["labels"]

    x_test = ds.test["inputs"].astype("float32")
    y_test = ds.test["labels"]

    # reshape to (28,28,1)
    x_train = x_train.reshape((-1, 28, 28, 1))
    x_test = x_test.reshape((-1, 28, 28, 1))

    return (x_train, y_train), (x_test, y_test)

In [ ]:
# ----------------------------
# Convolutional Neural Network
# ----------------------------
"""
Input layer with shape (28, 28, 1).
First convolutional block.
Second convolutional block.
Classifier head with 10-class output.
"""
def build_cnn_model(input_shape=(28, 28, 1), num_classes=10):
    model = models.Sequential(
        [
            layers.Input(shape=input_shape),

            layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
            layers.MaxPooling2D((2, 2)),

            layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
            layers.MaxPooling2D((2, 2)),

            layers.Flatten(),
            layers.Dense(64, activation="relu"),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model

In [ ]:
# ----------------------------
# Main
# ----------------------------
def main():
    (x_train, y_train), (x_test, y_test) = load_binarized_mnist()

    model = build_cnn_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    model.summary()

    # Early stopping on validation loss
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=3, restore_best_weights=True
        )
    ]

    history = model.fit(
        x_train,
        y_train,
        epochs=20,
        batch_size=128,
        validation_split=0.1,
        callbacks=callbacks,
        verbose=2,
    )

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"CNN model – Test accuracy: {test_acc:.4f}, loss: {test_loss:.4f}")

In [ ]:
if __name__ == "__main__":
    main()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 220,234 (860.29 KB)

 Trainable params: 220,234 (860.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
352/352 - 51s - 145ms/step - accuracy: 0.9217 - loss: 0.2674 - val_accuracy: 0.9614 - val_loss: 0.1163
Epoch 2/20
352/352 - 50s - 142ms/step - accuracy: 0.9776 - loss: 0.0716 - val_accuracy: 0.9732 - val_loss: 0.0778
Epoch 3/20
352/352 - 53s - 151ms/step - accuracy: 0.9847 - loss: 0.0493 - val_accuracy: 0.9748 - val_loss: 0.0780
Epoch 4/20
352/352 - 77s - 218ms/step - accuracy: 0.9882 - loss: 0.0378 - val_accuracy: 0.9794 - val_loss: 0.0630
Epoch 5/20
352/352 - 50s - 141ms/step - accuracy: 0.9915 - loss: 0.0267 - val_accuracy: 0.9798 - val_loss: 0.0645
Epoch 6/20
352/352 - 79s - 226ms/step - accuracy: 0.9929 - loss: 0.0225 - val_accuracy: 0.9786 - val_loss: 0.0675
Epoch 7/20
352/352 - 83s - 236ms/step - accuracy: 0.9942 - loss: 0.0182 - val_accuracy: 0.9788 - val_loss: 0.0789
CNN model – Test accuracy: 0.9877, loss: 0.0459
